# component magnitude checks


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cmocean
import netCDF4 as nc
from itertools import compress
import datetime as dt
import os
import sys

%matplotlib notebook

Load domain dimensions

In [6]:
imin, imax = 1480, 2180
jmin, jmax = 160, 800

In [7]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12_Ariane_mesh.nc')
tmask = np.array(mask.variables['tmask'])
cond  = (tmask[0,:,:,:] > 0.1) 
land_mask = np.ma.masked_where(cond, tmask[0,:,:,:]) 
tmask     = tmask[0,:,imin:imax,jmin:jmax]

In [9]:
# Load structure from sample dataset:
ref   = nc.Dataset('/data/brogalla/ANHA12/ANHA12-EXH006_5d_gridT_y2015m01d05.nc',  'r')
lat_model = ref.variables['nav_lat'][imin:imax, jmin:jmax]
lon_model = ref.variables['nav_lon'][imin:imax, jmin:jmax]

Volume of grid cells

In [13]:
e1t_base = np.array(mask.variables['e1t'])[0,imin:imax,jmin:jmax]
e2t_base = np.array(mask.variables['e2t'])[0,imin:imax,jmin:jmax]
e3t = np.array(mask.variables['e3t_0'])[0,:,imin:imax,jmin:jmax]

e1t    = np.tile(e1t_base, (50,1,1))
e2t    = np.tile(e2t_base, (50,1,1))

volume3d  = e1t*e2t*e3t
area      = e1t_base*e2t_base

Constants

In [17]:
alpha   = 0.65   # dimensionless
f_mnatm = 0.527  # g of Mn / kg
f_mnsed = 0.274  # g of Mn / kg
f_mnice = 0.170  # g of Mn / kg
molar_m = 54.938 # g of Mn / mol

R_class_max = 165e-9*1e3 # mol/m3
R_class_ave = 30e-9*1e3  # mol/m3

discharge_max = 0.0558*3600*24*365 # kg/m2/yr
discharge_ave = 0.0014*3600*24*365 # kg/m2/yr

erosion_max = 1.74e-06 # dimensionless
erosion_ave = 6.69e-09 # dimensionless

flx_atm_max = 2.27e-3*1e-3 # kg/m2/yr 
flx_atm_ave = 9.87e-5*1e-3 # kg/m2/yr

flx_ice_max = 5.15*1e-3 # kg/m2/yr
flx_ice_ave = 0.11*1e-3 # kg/m2/yr

##### Rivers:

\begin{equation}
\frac{\partial [dMn]}{\partial t} \bigg\vert_{river} = \frac{ Q \cdot R_{class}}{\rho_0 \ \Delta z_{surface}}
\end{equation}

Contribution to surface
\begin{equation}
\frac{\mu mol}{m^2\ year} = \frac{ Q \cdot R_{class}}{\rho_0}
\end{equation}

In [29]:
riv_ave = discharge_ave*R_class_ave*1e6/1000
riv_max = discharge_max*R_class_max*1e6/1000
print('Ave: ', "{0:.2}".format(riv_ave))
print('Max: ', "{0:.2}".format(riv_max))

Ave:  1.3e+03
Max:  2.9e+05


##### Dust from atmosphere:

\begin{equation}
    \frac{\partial [dMn]}{\partial t} \bigg\vert_{atmosphere} = \frac{\alpha \cdot f_{Mn,\ atm}}{m \cdot \Delta z_{surface}} \cdot \Phi_{atmosphere}
\end{equation}

Contribution to surface
\begin{equation}
\frac{\mu mol}{m^2\ year} = \frac{\alpha \cdot f_{Mn,\ atm}}{m} \cdot \Phi_{atmosphere}
\end{equation}

In [30]:
atm_ave = alpha*f_mnatm*flx_atm_ave*1e6/molar_m
atm_max = alpha*f_mnatm*flx_atm_max*1e6/molar_m
print('Ave: ', "{0:.2}".format(atm_ave))
print('Max: ', "{0:.2}".format(atm_max))

Ave:  0.00062
Max:  0.014


##### Dust from sea ice:

\begin{equation}
    \frac{\partial [dMn]}{\partial t} \bigg\vert_{ice} = \frac{\alpha \cdot f_{Mn,\ ice}}{m \cdot \Delta z_{surface}} \cdot \Phi_{ice}
\end{equation}

Contribution to surface
\begin{equation}
\frac{\mu mol}{m^2\ year} = \frac{\alpha \cdot f_{Mn,\ ice}}{m} \cdot \Phi_{ice}
\end{equation}

In [31]:
ice_ave = alpha*f_mnice*flx_ice_ave*1e6/molar_m
ice_max = alpha*f_mnice*flx_ice_max*1e6/molar_m
print('Ave: ', "{0:.2}".format(ice_ave))
print('Max: ', "{0:.2}".format(ice_max))

Ave:  0.22
Max:  1e+01


##### Resuspended sediment

\begin{equation}
    \frac{\partial [dMn]}{\partial t} \bigg\vert_{sediment} = C \cdot U_{tidal}^2 \cdot \frac{\alpha \cdot f_{Mn,\ sed}}{m \cdot \Delta z_{bottom}}
\end{equation}

Contribution to surface
\begin{equation}
\frac{\mu mol}{m^2\ year} = C \cdot U_{tidal}^2 \cdot \frac{\alpha \cdot f_{Mn,\ sed}}{m}
\end{equation}

In [32]:
sed_ave = erosion_ave*alpha*f_mnsed*1e6/molar_m
sed_max = erosion_max*alpha*f_mnsed*1e6/molar_m
print('Ave: ', "{0:.2}".format(sed_ave))
print('Max: ', "{0:.2}".format(sed_max))

Ave:  2.2e-05
Max:  0.0056


## Component comparison

|    Component          | Van Hulten model         | My model average          | My model maximum          |
|:----------------------|:-------------------------|:--------------------------|:--------------------------|
| Mn from atmosphere    | 0-3 $\mu$mol/m$^2$/yr    | 6.2e-4 $\mu$mol/m$^2$/yr  | 0.014  $\mu$mol/m$^2$/yr  | 
| Mn from ice           |      NA                  | 0.22   $\mu$mol/m$^2$/yr  | 10     $\mu$mol/m$^2$/yr  |
| Mn from sediments     | 0-70 $\mu$mol/m$^2$/yr   | 2.2e-5 $\mu$mol/m$^2$/yr  | 5.6e-3 $\mu$mol/m$^2$/yr  |